In [60]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas

In [61]:
# Basic Request
#player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)
#player_info = commonplayerinfo.player_headline_stats(player_id=2544)
#playerdashboard = playerdashboardbyteamperformance.PlayerDashboardByTeamPerformance(player_id=2544)
#print(playerdashboard.overall_player_dashboard.get_json())

player1_input = input('Enter player 1: ')
print(player1_input)

player1 = players.find_players_by_full_name(player1_input)[0]['id']

player_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=player1)
player_headers = player_info.overall_player_dashboard.get_dict().get('headers')
player_data = player_info.overall_player_dashboard.get_dict().get('data')[0]

###last 20 games###
#player_headers = player_info.last20_player_dashboard.get_dict().get('headers')
#player_data = player_info.last20_player_dashboard.get_dict().get('data')[0]

print(player_headers[5], player_data[5], 
	player_headers[2], player_data[2], 
	player_headers[26], player_data[26],
	player_headers[27], player_data[27])

Enter player 1: lebron james
lebron james
W_PCT 0.509 GP 55 PTS 1505 PLUS_MINUS 116


In [62]:
team1_input = input('Enter team 1: ')
print(team1_input)

team1 = teams.find_teams_by_full_name(team1_input)[0]['id']
print(team1)
teamdashboard = teamdashboardbyteamperformance.TeamDashboardByTeamPerformance(team_id=team1)
team_headers = teamdashboard.overall_team_dashboard.get_dict().get('headers')
team_data = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]

print('Season: ', team_data[-1])
print(team_headers[9], ': ', team_data[9])

Enter team 1: lakers
lakers
1610612747
Season:  2018-19
FG_PCT :  0.47


In [63]:
###PER stats (2018-19)####
lg_AST = 24.6
lg_FG = 41.1
lg_PTS = 111.2
lg_FGA = 89.2
lg_FT = 17.7
lg_FTA = 23.1
lg_TRB = 45.2
lg_ORB = 10.3
lg_TOV = 14.1
lg_PF = 20.9
lg_pace = 100

factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
DRB_perc = (lg_TRB -lg_ORB) / lg_TRB
 
print('league stats:', factor, VOP, DRB_perc)

#####PER player stats####
MP = player_data[6]
ThrP = player_data[10] 
AST = player_data[19]
TOV = player_data[20]
FGA = player_data[8]
FG = player_data[7] + player_data[10] 
FTA = player_data[14]
FT = player_data[13]
TRB = player_data[18]
ORB = player_data[16]
STL = player_data[21]
BLK = player_data[22]
PF = player_data[24]
print(MP)

####PER team stats####
team_AST = team_data[19]
team_FG = team_data[7]+team_data[10]
team_pace = 103.2 #Lakers
print(team_AST, team_FG)

league stats: 0.6022253005842968 1.0778953898646815 0.7721238938053098
1937.4016666666666
2096 4338


In [64]:
####PER Calculation####                    
PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST
     + (2 - factor * (team_AST / team_FG)) * FG
     + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
     - VOP * TOV - VOP * DRB_perc * (FGA - FG)
     - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
     + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB + VOP * STL + VOP * DRB_perc * BLK - PF * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)


print(PER_calc_player, adj_PER_calc_player)

0.727878630589535 0.7053087505712548


In [16]:
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=1).get_dict()#.PlayersSeasonTotals(team_id=team1, last_n_games=20)
print(roster1)

{'resource': 'teamplayerdashboard', 'parameters': {'MeasureType': 'Base', 'PerMode': 'Totals', 'PlusMinus': 'N', 'PaceAdjust': 'N', 'Rank': 'N', 'LeagueID': None, 'Season': '2018-19', 'SeasonType': 'Regular Season', 'PORound': None, 'TeamID': 1610612747, 'Outcome': None, 'Location': None, 'Month': 0, 'SeasonSegment': None, 'DateFrom': None, 'DateTo': None, 'OpponentTeamID': 0, 'VsConference': None, 'VsDivision': None, 'GameSegment': None, 'Period': 0, 'ShotClockRange': None, 'LastNGames': 20}, 'resultSets': [{'name': 'TeamOverall', 'headers': ['GROUP_SET', 'TEAM_ID', 'TEAM_NAME', 'GROUP_VALUE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RA